In [ ]:
import requests
import json
import pandas as pd
from datetime import date, timedelta
import os
import csv

In [ ]:
stock_path = ['Stock_Ratio.csv']
for a in stock_path:
    if os.path.exists(a):
        os.remove(a)

In [ ]:
def get_web(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.27 Safari/537.36 Edg/99.0.1150.16'}
    response = requests.get(url, headers=headers)
    return response

In [ ]:
def get_stock_ratio(stock_code, report_date):
    url = 'https://finfo-api.vndirect.com.vn/v4/ratios?q=group:stock~code:' + stock_code + '~reportDate:' + report_date
    data = get_web(url).json()
    page_size = data['totalElements']
    
    #Resize array
    url = url + '&sort=reportDate~code:asc~itemCode&size=' + str(page_size)
    data = get_web(url).json()

    stock_ratio_list = []
    for d in data['data']:
        try:
            date_time = d['reportDate']
            stock_type = d['group']
            stock_id = d['code']
            item_id = d['itemCode']
            item_name = d['itemName']
            value = d['value']

            stock_ratio_list.append([date_time, stock_type, stock_id, item_id, item_name, value])
        except Exception as err:
            print(err)
    
    return stock_ratio_list

In [ ]:
def get_trading_day_eoq():
    url = 'https://finfo-api.vndirect.com.vn/v4/trading_calendars?q=endDayOfQuarter:true'
    data = get_web(url).json()
    page_size = data['totalElements']
    
    #Resize array
    url = url + '&sort=date&size=' + str(page_size)
    data = get_web(url).json()
    
    trading_day_eoq_list = []
    for d in data['data']:
        try:
            date = d['date']

            trading_day_eoq_list.append(date)
        except Exception as err:
            print(err)
    
    return trading_day_eoq_list

In [ ]:
%%time

detail = []
floor = ['HOSE', 'HNX', 'UPCOM']
for f in floor: #instead of doing this, you can get a request again
    # Prep stock code
    df = pd.read_csv('Stock_List.csv')
    stock_list = df.loc[(df['stock_floor']==f)]['stock_code'].to_list()
    stock_code = ''
    for s in stock_list:
        stock_code = s + ',' + stock_code
        
    # Prep report date
    #date_list = ['2021-12-31']
    start_date = date(2021,12,31).strftime('%Y-%m-%d')
    end_date = date(2021,12,31).strftime('%Y-%m-%d')
    #date_list = pd.date_range(start_date, end_date-timedelta(days=0), freq='d').strftime('%Y-%m-%d').to_list()
    date_list = [dt for dt in get_trading_day_eoq() if start_date <= dt <= end_date]
    report_date = ''
    for d in date_list:
        report_date = d + ',' + report_date
        
    data = get_stock_ratio(stock_code, report_date)
    for d in data:
        detail.append(d)

# Prep csv
header = [['date_time', 'stock_type', 'stock_code', 'item_id', 'item_name', 'value']]
with open(stock_path[0], 'w', newline='', encoding='utf-8-sig') as f:
    write = csv.writer(f)
    write.writerows(header)
    write.writerows(detail)
    
print('Crawl done!')